In [ ]:
pip install -q -r requirements.txt

In [ ]:
import logging
import json
from IPython.display import Pretty, HTML
import urllib3

from FhirClient import FhirClient

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

# Disable the unverified HTTPS request warnings
urllib3.disable_warnings()

In [ ]:
def setup_fhir_client() -> FhirClient:
    fhir_client = FhirClient(
        base_url='https://concept01.ehealthexchange.org:52780/fhirproxy/r4/',
        verify_ssl=False,
        extra_headers={
            'X-DESTINATION': 'PublicHAPI',
            'X-POU': 'TREAT',
            'Host': 'concept01.ehealthexchange.org:52780'
        })

    fhir_client.auth_type = 'Basic'
    fhir_client.token = 'c3ZjX2VIeEZISVJTYW5kYm94OndpbGxmdWxTdHJvbmdTdGFuZHVyZDc'

    return fhir_client

In [ ]:
client = setup_fhir_client()

search_results = client.search('Patient', {
    'family': 'Sanity',
    'gender': 'male',
    'birthdate': '1919-11-03',
    'address-state': 'MH',
})
Pretty(json.dumps(search_results, indent=4))

There should be exactly one `Patient` matching the search.

In [ ]:
patient = search_results['entry'][0]
Pretty(json.dumps(patient, indent=4))

In [ ]:
HTML(patient['resource']['text']['div'])

`MedicationAdministration` should have the medications administered to this Patient.

In [ ]:
medication_administration_results = client.search('MedicationAdministration', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(medication_administration_results, indent=4))

`Observation` should have lab test results, and pregnancy status.

In [ ]:
observation_search_results = client.search('Observation', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(observation_search_results, indent=4))

`Encounter` should have hospitalization status.

In [ ]:
encounter_search_results = client.search('Encounter', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(encounter_search_results, indent=4))

In [ ]:
client.close()